In [1]:
import pandas as pd
import time
import datetime
import requests
import traceback
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, Float, String, DateTime
import sqlalchemy
import numpy as np
from datetime import datetime
import mysql.connector

# hide ipykernel warnings 
import warnings
warnings.filterwarnings('ignore')

In [31]:
engine = create_engine(f"mysql+mysqlconnector://{name}:{password}@{host}:3306/{db}", echo=True)

In [32]:
db = mysql.connector.connect(
    host=myhost,
    user=myuser,
    password=mypasswd,
    database=mydatabase
)

cur = db.cursor()

In [33]:
engine = create_engine(f"mysql+mysqlconnector://{myuser}:{mypasswd}@{myhost}:3306/{mydatabase}", echo=True)
connection = engine.connect()

2021-08-09 16:25:37,586 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2021-08-09 16:25:37,586 INFO sqlalchemy.engine.base.Engine {}
2021-08-09 16:25:37,966 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2021-08-09 16:25:37,967 INFO sqlalchemy.engine.base.Engine {}
2021-08-09 16:25:38,717 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2021-08-09 16:25:38,718 INFO sqlalchemy.engine.base.Engine {}
2021-08-09 16:25:39,476 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2021-08-09 16:25:39,476 INFO sqlalchemy.engine.base.Engine {}
2021-08-09 16:25:39,861 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2021-08-09 16:25:39,863 INFO sqlalchemy.engine.base.Engine {}


<h3>routes.txt</h3>
<p>Find all currently operational lines (GTFS data last downloaded on 02/08/21)</p>

In [2]:
lines_df = pd.read_csv("routes.txt")

In [3]:
lines_df.head()

,route_id,agency_id,route_short_name,route_long_name,route_type
0,60-1-b12-1,978,1,NaN,3
1,60-102-b12-1,978,102,NaN,3
2,60-104-b12-1,978,104,NaN,3
3,60-11-b12-1,978,11,NaN,3
4,60-111-b12-1,978,111,NaN,3


In [4]:
# List currently operational lines
all_lines = lines_df.route_short_name.unique().tolist()
all_lines.sort()
print(all_lines)

['1', '102', '104', '11', '111', '114', '116', '118', '120', '122', '123', '13', '130', '14', '140', '142', '145', '14c', '15', '150', '151', '155', '15a', '15b', '15d', '16', '161', '16c', '16d', '17', '17a', '18', '184', '185', '220', '236', '238', '239', '25', '25a', '25b', '25d', '25x', '26', '27', '270', '27a', '27b', '27x', '29a', '31', '31a', '31b', '31d', '32', '32x', '33', '33a', '33b', '33d', '33e', '33x', '37', '38', '38a', '38b', '38d', '39', '39a', '39x', '4', '40', '40b', '40d', '40e', '41', '41a', '41b', '41c', '41d', '41x', '42', '42d', '43', '44', '44b', '451', '45a', '46a', '46e', '47', '49', '51d', '51x', '53', '53a', '54a', '56a', '59', '6', '61', '63', '65', '65b', '66', '66a', '66b', '66e', '66x', '67', '67x', '68', '68a', '68x', '69', '69x', '7', '70', '70d', '747', '75', '757', '76', '76a', '77a', '77x', '79', '79a', '7a', '7b', '7d', '8', '83', '83a', '84', '84a', '84x', '86', '9', 'H1', 'H2', 'H3', 'H9']


In [5]:
len(all_lines)

143

In [6]:
line_ids = lines_df[["route_id", "route_short_name"]].copy()

In [7]:
line_ids.rename(columns={"route_short_name": "line"}, inplace=True)

In [8]:
line_ids.head()

,route_id,line
0,60-1-b12-1,1
1,60-102-b12-1,102
2,60-104-b12-1,104
3,60-11-b12-1,11
4,60-111-b12-1,111


<h3>shapes.txt</h3>
<p>Data describing the path that each route takes. N.B. coordinates do not represent stops on the route.</p>

In [9]:
shapes_df = pd.read_csv("shapes.txt")

In [10]:
shapes_df.head()

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
0,60-116-b12-1.56.O,53.329631,-6.249012,1,0.00
1,60-116-b12-1.56.O,53.329316,-6.248409,2,53.15
2,60-116-b12-1.56.O,53.329056,-6.247819,3,101.85
3,60-116-b12-1.56.O,53.328519,-6.246715,4,196.45
4,60-116-b12-1.56.O,53.328406,-6.246390,5,221.44


In [11]:
shapes_df["line"] = shapes_df.apply(lambda row: row.shape_id.split("-")[1], axis=1)

In [12]:
shapes_df.head()

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled,line
0,60-116-b12-1.56.O,53.329631,-6.249012,1,0.00,116
1,60-116-b12-1.56.O,53.329316,-6.248409,2,53.15,116
2,60-116-b12-1.56.O,53.329056,-6.247819,3,101.85,116
3,60-116-b12-1.56.O,53.328519,-6.246715,4,196.45,116
4,60-116-b12-1.56.O,53.328406,-6.246390,5,221.44,116


In [13]:
shapes_df["route_id"] = shapes_df.apply(lambda row: row.shape_id.split(".")[0], axis=1)

In [14]:
shapes_df.head()

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled,line,route_id
0,60-116-b12-1.56.O,53.329631,-6.249012,1,0.00,116,60-116-b12-1
1,60-116-b12-1.56.O,53.329316,-6.248409,2,53.15,116,60-116-b12-1
2,60-116-b12-1.56.O,53.329056,-6.247819,3,101.85,116,60-116-b12-1
3,60-116-b12-1.56.O,53.328519,-6.246715,4,196.45,116,60-116-b12-1
4,60-116-b12-1.56.O,53.328406,-6.246390,5,221.44,116,60-116-b12-1


In [15]:
shapes_df.shape

(251080, 7)

In [16]:
shape_ids = shapes_df.shape_id.unique().tolist()

In [17]:
# Number of routes
len(shape_ids)

719

In [18]:
lines = shapes_df.line.unique().tolist()
lines.sort()

In [19]:
# Number of lines
len(lines)

102

In [20]:
print(lines)

['1', '11', '116', '118', '120', '122', '123', '13', '130', '14', '140', '142', '145', '15', '150', '151', '155', '15A', '15B', '15D', '16', '16D', '25', '25A', '25B', '25D', '25X', '26', '27', '27A', '27B', '27X', '32X', '33', '33D', '33E', '33X', '37', '38', '38A', '38B', '38D', '39', '39A', '39X', '4', '40', '40B', '40D', '40E', '41', '41B', '41C', '41D', '41X', '42', '43', '44', '44B', '46A', '46E', '47', '49', '51D', '53', '54A', '56A', '6', '61', '65', '65B', '66', '66A', '66B', '66E', '66X', '67', '67X', '68', '68A', '68X', '69', '69X', '7', '70', '77A', '77X', '79', '79A', '7A', '7B', '7D', '83', '83A', '84', '84A', '84X', '9', 'H1', 'H2', 'H3', 'H9']


<p>Dropping shape_pt data as these do not necessarily relate to bus stops and it is bus stops only that we want to plot</p>

In [21]:
shapes_df.drop(["shape_pt_lat", "shape_pt_lon", "shape_pt_sequence", "shape_dist_traveled"], axis=1, inplace=True)

In [22]:
shapes_df.head()

,shape_id,line,route_id
0,60-116-b12-1.56.O,116,60-116-b12-1
1,60-116-b12-1.56.O,116,60-116-b12-1
2,60-116-b12-1.56.O,116,60-116-b12-1
3,60-116-b12-1.56.O,116,60-116-b12-1
4,60-116-b12-1.56.O,116,60-116-b12-1


In [23]:
shapes_df.shape

(251080, 3)

In [24]:
shapes_df.drop_duplicates(inplace=True)

In [25]:
shapes_df.shape

(719, 3)

In [26]:
shapes_df.head()

,shape_id,line,route_id
0,60-116-b12-1.56.O,116,60-116-b12-1
273,60-116-b12-1.57.I,116,60-116-b12-1
656,60-116-d12-1.55.O,116,60-116-d12-1
929,60-116-d12-1.56.I,116,60-116-d12-1
1312,60-118-b12-1.58.I,118,60-118-b12-1


<h3>Review Trips to find destination for a journey</h3>

In [27]:
trips_df = pd.read_csv("trips.txt")

In [28]:
trips_df.head()

,route_id,service_id,trip_id,shape_id,trip_headsign,direction_id
0,60-116-b12-1,y1003,3736.y1003.60-116-b12-1.57.I,60-116-b12-1.57.I,Church of Our Lady of Good Council - Parnell S...,1
1,60-116-b12-1,y1003,3737.y1003.60-116-b12-1.56.O,60-116-b12-1.56.O,Sussex Road - Church of Our Lady of Good Council,0
2,60-116-d12-1,y1007,3736.y1007.60-116-d12-1.56.I,60-116-d12-1.56.I,Church of Our Lady of Good Council - Parnell S...,1
3,60-116-d12-1,y1007,3737.y1007.60-116-d12-1.55.O,60-116-d12-1.55.O,Sussex Road - Church of Our Lady of Good Council,0
4,60-118-b12-1,y1003,3770.y1003.60-118-b12-1.58.I,60-118-b12-1.58.I,Enniskerry Road (Ballycorus Rd) - Eden Quay,1


In [29]:
trips_df.drop(["service_id", "trip_id", "shape_id"], axis=1, inplace=True)

In [30]:
trips_df.head()

,route_id,trip_headsign,direction_id
0,60-116-b12-1,Church of Our Lady of Good Council - Parnell S...,1
1,60-116-b12-1,Sussex Road - Church of Our Lady of Good Council,0
2,60-116-d12-1,Church of Our Lady of Good Council - Parnell S...,1
3,60-116-d12-1,Sussex Road - Church of Our Lady of Good Council,0
4,60-118-b12-1,Enniskerry Road (Ballycorus Rd) - Eden Quay,1


In [31]:
trips_df.shape

(32683, 3)

In [32]:
trips_df.drop_duplicates(inplace=True)

In [33]:
trips_df.shape

(394, 3)

In [34]:
trips_lines_merged = trips_df.merge(line_ids, how="left", on="route_id")

In [35]:
trips_lines_merged.head()

,route_id,trip_headsign,direction_id,line
0,60-116-b12-1,Church of Our Lady of Good Council - Parnell S...,1,116
1,60-116-b12-1,Sussex Road - Church of Our Lady of Good Council,0,116
2,60-116-d12-1,Church of Our Lady of Good Council - Parnell S...,1,116
3,60-116-d12-1,Sussex Road - Church of Our Lady of Good Council,0,116
4,60-118-b12-1,Enniskerry Road (Ballycorus Rd) - Eden Quay,1,118


In [36]:
trips_lines_merged.shape

(394, 4)

In [37]:
trips_lines_merged.isna().sum()

route_id         0
trip_headsign    0
direction_id     0
line             0
dtype: int64

<h3>Find stops per route using stop_times.txt</h3>

In [41]:
stop_times_df = pd.read_csv("stop_times.txt")

In [42]:
stop_times_df.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,7712.y1009.60-1-d12-1.1.O,18:40:00,18:40:00,8240DB000226,1,Sandymount,0,0,0.00
1,7712.y1009.60-1-d12-1.1.O,18:40:44,18:40:44,8220DB000228,2,Sandymount,0,0,267.48
2,7712.y1009.60-1-d12-1.1.O,18:41:21,18:41:21,8240DB000229,3,Sandymount,0,0,483.53
3,7712.y1009.60-1-d12-1.1.O,18:42:20,18:42:20,8240DB000227,4,Sandymount,0,0,834.47
4,7712.y1009.60-1-d12-1.1.O,18:42:58,18:42:58,8240DB000230,5,Sandymount,0,0,1063.49


In [43]:
stop_times_df.drop_off_type.unique()

array([0, 1], dtype=int64)

In [44]:
stop_times_df.pickup_type.unique()

array([0, 1], dtype=int64)

In [45]:
stop_times_df.drop(["pickup_type", "drop_off_type"], axis=1, inplace=True)

In [46]:
stop_times_df.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,shape_dist_traveled
0,7712.y1009.60-1-d12-1.1.O,18:40:00,18:40:00,8240DB000226,1,Sandymount,0.00
1,7712.y1009.60-1-d12-1.1.O,18:40:44,18:40:44,8220DB000228,2,Sandymount,267.48
2,7712.y1009.60-1-d12-1.1.O,18:41:21,18:41:21,8240DB000229,3,Sandymount,483.53
3,7712.y1009.60-1-d12-1.1.O,18:42:20,18:42:20,8240DB000227,4,Sandymount,834.47
4,7712.y1009.60-1-d12-1.1.O,18:42:58,18:42:58,8240DB000230,5,Sandymount,1063.49


In [47]:
stop_times_df["route_id"] = stop_times_df.apply(lambda row: row.trip_id.split(".")[2], axis=1)

In [48]:
stop_times_df.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,shape_dist_traveled,route_id
0,7712.y1009.60-1-d12-1.1.O,18:40:00,18:40:00,8240DB000226,1,Sandymount,0.00,60-1-d12-1
1,7712.y1009.60-1-d12-1.1.O,18:40:44,18:40:44,8220DB000228,2,Sandymount,267.48,60-1-d12-1
2,7712.y1009.60-1-d12-1.1.O,18:41:21,18:41:21,8240DB000229,3,Sandymount,483.53,60-1-d12-1
3,7712.y1009.60-1-d12-1.1.O,18:42:20,18:42:20,8240DB000227,4,Sandymount,834.47,60-1-d12-1
4,7712.y1009.60-1-d12-1.1.O,18:42:58,18:42:58,8240DB000230,5,Sandymount,1063.49,60-1-d12-1


In [49]:
stop_times_df.shape

(1874919, 8)

In [50]:
stop_times_df.drop(["trip_id", "arrival_time", "departure_time", "stop_headsign", "shape_dist_traveled"], axis=1, inplace=True)

In [51]:
stop_times_df.head()

,stop_id,stop_sequence,route_id
0,8240DB000226,1,60-1-d12-1
1,8220DB000228,2,60-1-d12-1
2,8240DB000229,3,60-1-d12-1
3,8240DB000227,4,60-1-d12-1
4,8240DB000230,5,60-1-d12-1


In [52]:
stop_times_df["stop_short"] = stop_times_df.apply(lambda row: row.stop_id[-4:], axis=1)

In [53]:
stop_times_df.drop(["stop_id"], axis=1, inplace=True)

In [54]:
stop_times_df.head()

,stop_sequence,route_id,stop_short
0,1,60-1-d12-1,0226
1,2,60-1-d12-1,0228
2,3,60-1-d12-1,0229
3,4,60-1-d12-1,0227
4,5,60-1-d12-1,0230


In [55]:
stop_times_df.drop_duplicates(inplace=True)

In [56]:
stop_times_df.shape

(27203, 3)

In [57]:
stop_times_merged = stop_times_df.merge(trips_lines_merged, how="left", on="route_id")

In [58]:
stop_times_merged.head()

,stop_sequence,route_id,stop_short,trip_headsign,direction_id,line
0,1,60-1-d12-1,0226,Shanard Road (Shanard Avenue) - Saint John's C...,0,1
1,1,60-1-d12-1,0226,Saint John's Church - Shanard Road (Shanard Av...,1,1
2,2,60-1-d12-1,0228,Shanard Road (Shanard Avenue) - Saint John's C...,0,1
3,2,60-1-d12-1,0228,Saint John's Church - Shanard Road (Shanard Av...,1,1
4,3,60-1-d12-1,0229,Shanard Road (Shanard Avenue) - Saint John's C...,0,1


<h3>Add in stop information</h3>

In [59]:
stops_df = pd.read_csv("stops.txt")

In [60]:
stops_df.head()

,stop_id,stop_name,stop_lat,stop_lon
0,8220DB000002,"Parnell Square West, stop 2",53.352244,-6.263723
1,8220DB000003,"Parnell Square West, stop 3",53.352309,-6.263811
2,8220DB000004,"Parnell Square West, stop 4",53.352575,-6.264175
3,8220DB000006,"Parnell Square West, stop 6",53.352749,-6.264454
4,8220DB000007,"Parnell Square West, stop 7",53.352841,-6.264570


In [61]:
stops_df.shape

(4208, 4)

In [ ]:
def create_and_fill_table(df):
    meta = MetaData()
    
    table = Table(
        "lines_for_plotting", meta, 
        Column("stop_short", Integer),
        Column("line", String(12)),
        Column("route_endpoint", String(500)),
        Column("direction_id", Integer),
        Column("stop_name", String(500)),
        Column("stop_lat", Float),
        Column("stop_lon", Float)
    )
    
    meta.create_all(engine)
    df.to_sql("lines_for_plotting", con = engine, if_exists = "append", index=False, chunksize = 100)

In [ ]:
create_and_fill_table(merged)